Validate Plan

In [ ]:
plan = {
  "entities": [
    {
      "name": "StoreProduct",
      "arguments": {
        "product_name": "str",
        "currency": "str",
        "price": "float"
      }
    }
  ],
  "apps": [
    {
      "appName": "app",
      "kafka_brokers": {
        "localhost": {
          "url": "localhost",
          "description": "local development kafka broker",
          "port": 9092
        }
      },
      "title": "FastKafka App",
      "consumes_functions": {
        "on_store_product": {
          "topic": "store_product",
          "prefix": "on",
          "input_parameters": [
            {
              "msg": "StoreProduct"
            }
          ]
        }
      },
      "produces_functions": {
        "change_currency": {
          "topic": "change_currency",
          "prefix": "to",
          "input_parameters": [
            {
              "store_product": "StoreProduct"
            }
          ],
          "returns": "StoreProduct"
        }
      }
    }
  ]
}

In [ ]:
from typing import List, Dict, Any

def validate_entities(entities: List[Dict[str, Any]]) -> None:
    assert len(entities) > 0
    
    for entity in entities:
        for attribute_name in ["name", "arguments"]:
            assert attribute_name in entity.keys()


In [ ]:
validate_entities(plan["entities"])

In [ ]:
def validate_function(function: Dict[str, Any], is_produces_function: bool=False):
    function_keys = function.keys()
    for attribute_name in ["topic", "prefix", "input_parameters"]:
        assert attribute_name in function_keys
        
    if is_produces_function:
        assert "returns" in function_keys
        assert function["returns"] is not None

In [ ]:
def vlidate_apps(apps: List[Dict[str, Any]]) -> None:
    assert len(apps) > 0
    
    for app in apps:
        # assert there is at least one consumes/produces function for each app
        assert (len(app["consumes_functions"]) > 0 or len(app["produces_functions"]) > 0)
        for attribute_name in ["appName", "kafka_brokers", "title"]:
            assert attribute_name in app.keys()
            
        for consumes in app["consumes_functions"].values():
            validate_function(consumes)
            
        for produces in app["produces_functions"].values():
            validate_function(produces)

In [ ]:
vlidate_apps(plan["apps"])

In [ ]:
import inspect
from guides.application_test import TestMsg
lines = inspect.getsource(TestMsg)
print(lines)

class TestMsg(BaseModel):
    msg: str = Field(...)



Validate generated app

In [ ]:
generated_app = """
from fastkafka import FastKafka
from pydantic import BaseModel, Field


kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    }
}

title = "FastKafka Application"

kafka_app = FastKafka(
    title=title,
    kafka_brokers=kafka_brokers,
)


class StoreProduct():
    product_name: str = Field(..., description="Name of the product")
    currency: str = Field(..., description="Currency")
    price: float


@kafka_app.consumes(prefix="on", topic="store_product")
async def on_store_product(msg: StoreProduct):
    await to_change_currency(msg)


@kafka_app.produces(prefix="to", topic="change_currency")
async def to_change_currency(store_product: StoreProduct) -> StoreProduct:
    # Producing logic
    if store_product.currency == "HRK":
        store_product.currency = "EUR"
        store_product.price /= 7.5
    
    return store_product
"""

In [ ]:
plan = {
  "entities": [
    {
      "name": "StoreProduct",
      "arguments": {
        "product_name": "str",
        "currency": "str",
        "price": "float"
      }
    }
  ],
  "apps": [
    {
      "appName": "app",
      "kafka_brokers": {
        "localhost": {
          "url": "localhost",
          "description": "local development kafka broker",
          "port": 9092
        }
      },
      "title": "FastKafka App",
      "consumes_functions": {
        "on_store_product": {
          "topic": "store_product",
          "prefix": "on",
          "input_parameters": [
            {
              "msg": "StoreProduct"            }
          ]
        }
      },
      "produces_functions": {
        "to_change_currency": {
          "topic": "change_currency",
          "prefix": "to",
          "input_parameters": [
            {
              "store_product": "StoreProduct"
            }
          ],
          "returns": "StoreProduct"
        }
      }
    }
  ]
}

In [ ]:
import tempfile
from importlib.machinery import SourceFileLoader
import inspect

# TODO: compare types in the Plan and generated script
# TODO: check/compare parameters in decorator (prefix, topic...)

def validate_generated_functions(apps, module: SourceFileLoader, fix_prompt: str, is_produces_function: bool=False):
    function_type = "produces" if is_produces_function else "consumes"
    
    for app in plan["apps"]:
        for function_name, attributes in app[function_type + "_functions"].items():
            # generated app must have all functions defined in the Plan
            if not hasattr(module, function_name):
                fix_prompt += f'Fix: {app["appName"]} is missing {function_type} function: {function_name}\n'

            attr = getattr(module, function_name)
            attr_signature_keys = list(inspect.signature(attr).parameters.keys())
            
            for input_parameter in attributes["input_parameters"]:
                for parameter_name, parameter_type in input_parameter.items():
                    # Generatet consumes and produces function must have all input parameters from the Plan
                    if parameter_name not in attr_signature_keys: 
                        fix_prompt += f'Fix: {function_type} function {function_name} is missing input parameter {parameter_name}\n'
                        
            if is_produces_function:
                lines = inspect.getsource(attr)
                # At the end of each produces function must be a return statement
                if "return" not in lines.strip().split("\n")[-1]:
                    fix_prompt += f'Fix: {function_type} function {function_name} MUST return some message\n'
                    
    return fix_prompt
       

In [ ]:
with tempfile.NamedTemporaryFile(mode='w+', suffix = '.py') as tmp:
    # Save generated application to *.py file
    tmp.write(generated_app)
    #tmp.seek(0)
    tmp.flush()
 
    # imports the module from the given path
    module_name = "application"
    fix_prompt = ""
    try:
        module = SourceFileLoader(module_name, tmp.name).load_module()
        fix_prompt = validate_generated_functions(plan["apps"], module, fix_prompt, is_produces_function=False)
        fix_prompt = validate_generated_functions(plan["apps"], module, fix_prompt, is_produces_function=True)
    # try to fix an error if we can't import the generated file
    # (if some libs / variables / classes are used but not defined...)
    except Exception as e:
        fix_prompt += f"Fix: python file isn't valid, please resolve the following error: {e}\n"
    
print(fix_prompt)